Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [125]:
from random import choices, random, randint
import lab9_lib

In [126]:
pop = 1
fitness = lab9_lib.make_problem(pop)
for n in range(pop):
    ind = choices([0, 1], k=50)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

01000100111100010100001001011111101011011111010100: 52.00%
1


### Local Search EA:
The algorithm runs for a specified number of generation, evolving the population to improve the best fitness.
In each generation:
- Parents are selected from the current population using the `select` method
- Cross-over is applied to the parents to generate the offspring
- Mutation is applied to the offspring to generate the mutated offspring
- The worst individuals (based on fitness) are replaced by the mutated offspring

To reduce the number of fitness calls, the algorithm keeps track of the stagnation of the best fitness. If the best fitness does not improve for a specified number of generations, the algorithm stops. Also, if the best fitness arrives at a certain threshold, the algorithm stops.

In [127]:
class LocalSearchEA:
    def __init__(self, fitness_function, population_size=10, num_generations=100, mutation_rate=0.1, max_stagnation=20):
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.num_generations = num_generations
        self.mutation_rate = mutation_rate
        self.max_stagnation = max_stagnation
    
    def initialize_population(self):
        k = 1000
        return [choices([0, 1], k=k) for _ in range(self.population_size)]
    
    def mutate(self, individual):
        return [1 - gene if random() < self.mutation_rate else gene for gene in individual]
    
    def crossover(self, parent1, parent2):
        k = randint(1, len(parent1) - 1)
        child1 = parent1[:k] + parent2[k:]
        child2 = parent2[:k] + parent1[k:]
        return child1, child2
    
    def select(self, population):
        return choices(population, k=2, weights=[self.fitness_function(individual) for individual in population])
    
    def run(self):
        population = self.initialize_population()
        best_individual = max(population, key=self.fitness_function)
        best_fitness = self.fitness_function(best_individual)
        stagnation_count = 0
        
        for generation in range(1, self.num_generations + 1):
            paren1, parent2 = self.select(population)
            child1, child2 = self.crossover(paren1, parent2)
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            # Replace worst individual with best child
            worst_individual = min(population, key=self.fitness_function)
            population.remove(worst_individual)
            population.extend([child1, child2])
            
            current_best_individual = max(population, key=self.fitness_function)
            current_best_fitness = self.fitness_function(current_best_individual)
            
            if current_best_fitness > best_fitness:
                best_individual = current_best_individual
                best_fitness = current_best_fitness
                stagnation_count = 0
            else:
                stagnation_count += 1
                
            if stagnation_count >= self.max_stagnation:
                print(f"Stopping early after {generation} generations due to stagnation.")
                break
                
            if best_fitness >= 0.99:
                print(f"Stopping early after {generation} generations due to fitness convergence.")
                break
        print(f"fitness: {best_fitness},\nfitness calls: {self.fitness_function.calls}")
    

### Parameters:
Parameters are set by default, could be changed to get better results. 
- MUTATION_RATE: probability of mutation per gene in an individual
- NUM_GENERATIONS: number of generations to run the algorithm for before stopping, coul be increase to better results, but it will take more time
- MAX_STAGNATION: number of generations to run the algorithm without improvement before stopping, could be increased to better results, but it will take more time

In [128]:
MUTATION_RATE = 0.15 # probability of mutation per gene in an individual
NUM_GENERATIONS = 1000 # number of generations to run the algorithm for before stopping, coul be increase to better results, but it will take more time
MAX_STAGNATION = 100 # number of generations to run the algorithm without improvement before stopping, could be increased to better results, but it will take more time

In [129]:
instances = 1
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 1 instance(s)
Stopping early after 274 generations due to stagnation.
fitness: 0.76,
fitness calls: 113575


In [130]:
instances = 2
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 2 instance(s)
Stopping early after 242 generations due to stagnation.
fitness: 0.72,
fitness calls: 89422


In [131]:
instances = 5
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 5 instance(s)
Stopping early after 119 generations due to stagnation.
fitness: 0.55,
fitness calls: 23092


In [132]:
instances = 10
print(f"Local Search evolutionary algorithm with {instances} instance(s)")
fitness = lab9_lib.make_problem(instances)
ea = LocalSearchEA(fitness, 
                   population_size=instances, 
                   num_generations=NUM_GENERATIONS, 
                   mutation_rate=MUTATION_RATE, 
                   max_stagnation=MAX_STAGNATION)
ea.run()

Local Search evolutionary algorithm with 10 instance(s)
Stopping early after 201 generations due to stagnation.
fitness: 0.473378,
fitness calls: 66743
